## Import modules

In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import tools_v2 as tools
import Gaussian_functions_v2 as gf
import loss_functions as lf

import ipywidgets as widgets

base_dir = "dir1/dir2/"      # Data will be saved here
sample = "sample"            # sample name, which will be included in file names to be saved


## Import TEM images

In [ ]:
import hyperspy.api as hs       # modules needed to import TEM images in a form of ndarray.

file1 = hs.load(base_dir + "file_name1.tif").data 
file2 = hs.load(base_dir + "file_name2.dm4").data 

TEM_array = file1              # TEM_array should receive the 2d array form of the TEM image

plt.imshow(TEM_array, cmap = "grey")              
print("image shape:", TEM_array.shape)

## Image processing

In [ ]:
rotate_angle = 0.0   # counter-clockwise rotation in deg

# Boundary information for cropping 

x = 100                            # x coordinate of upper left vertex of the cropped image in pixels
y = 100                            # y coordinate of upper left vertex of the cropped image in pixels
width = 2000                       # width of the cropped image in pixels
height = 2000                      # height of the cropped image in pixels

line_width = 2       # line width for a box showing the boundary for the croped image

normalized_maximum_intensity = 10          # processed image will have this maximum intensity
low_intensity_threshold = 0.1              
# intensity less than the (threshold* maximum intensity) will be cut off for noise suppression

im_analyzed, tf_im_analyzed = tools.image_preprocess(TEM_array, rotate_angle, 
                                                     normalized_maximum_intensity, low_intensity_threshold, 
                                                     x, y, width, height, line_width)
im_shape = im_analyzed.shape

# im_analyzed is 2-d numpy array of the processed image, and tf_im_analyzed is the tensor version of that.

im_analyzed_file_name = "im_analyzed"
np.save(base_dir + im_analyzed_file_name + "_" + sample, im_analyzed)



## Lattice parameter setting and check

In [ ]:
len_pix = 0.1             # length of one pixel in angstrom

x_lattice = 5.0                     # lateral lattice constant in angstrom
y_lattice = 5.0                     # vertical lattice constant in angstrom
x_off = 10                      # x coordinate of the upper left lattice point in pixel
y_off = 10                      # y coordinate of the upper left lattice point in pixel
sliding = [0,     #1
           0,     #2
           0,     #3
           0]     #4    
# Relative x offsets of the lattice points from the second to last rows with respect to that of the first row

row = 5                        # number of rows to be analyzed
col = 10                       # number of columns to be analyzed

x_look = 0                     # upper left x coordinate of focusing box in the unit cell in pixel
y_look = 0                     # upper left y coordinate of focusing box in the unit cell in pixel
width_look = 50                # width of focusing box in the unit cell
height_look = 40               # height of focusing box in the unit cell

unit_cell_image, tf_unit_cell_image, lattices, lattice_num, TEM_image_with_grid = \
tools.lattice_parameter_tunning(x_lattice, y_lattice, x_off, y_off, len_pix, row, col, sliding, im_analyzed,
                             x_look, y_look, width_look, height_look)

# unit_cell_image is a 2-dimensional numpy array of unit cell image.
# tf_unit_cell_image is the tensor version of unit_cell_image


lattice_num_file_name = "lattice_num"
unitcell_file_name = "unitcell"
x_lattice_file_name = "x_lattice"
y_lattice_file_name = "y_lattice"
len_pix_file_name = "len_pix"

np.save(base_dir + lattice_num_file_name + "_" + sample, lattice_num)
np.save(base_dir + unitcell_file_name + "_" + sample, unit_cell_image)
np.save(base_dir + x_lattice_file_name + "_" + sample, x_lattice)
np.save(base_dir + y_lattice_file_name + "_" + sample, y_lattice)
np.save(base_dir + len_pix_file_name + "_" + sample, len_pix)

TEM_image_with_grid.savefig(base_dir + f"TEM_image_with_grid_{sample}.png", dpi = 300, bbox_inches = 'tight')


## Unit cell investigator

In [ ]:
tools.unit_cell_investigator(unit_cell_image)

## Atomic positions in unit cell setting

In [ ]:
Atom_positions_dic = {"Atom 1" : [[0.1, 0.1], [0.9, 0.9],], 
                      "Atom 2": [[0.5,0.5],],
                      "Atom 3": [[0.4, 0.2], [0.6, 0.2],
                                [0.4, 0.8], [0.6, 0.8],]}

marker_size = 50

posit_pix, atom_num_list, atom_num_in_unit_cell = tools.atom_positions_iu_gen_check(Atom_positions_dic, 
                                                                                    unit_cell_image, 
                                                                                    x_lattice, 
                                                                                    y_lattice, 
                                                                                    len_pix,
                                                                                    marker_size)

atom_name_list = list(Atom_positions_dic.keys())

atom_num_list_file_name = "atom_num_list"
atom_name_list_file_name = "atom_name_list"
np.save(base_dir + atom_num_list_file_name + "_" + sample, atom_num_list)
np.save(base_dir + atom_name_list_file_name + "_" + sample, atom_name_list)

## Initialize atomic positions with peak finder unit cell

In [ ]:
# each element is for each atom in the keywords of Atom_positions_dic.

peak_finder_pad_size_list = [5, 5, 5]    

marker_size = 50

first_posit_pix_file_name = "first_posit_pix"

posit_pix_peaks = tools.unit_cell_peak_finder(peak_finder_pad_size_list, 
                          atom_num_list, posit_pix, unit_cell_image, marker_size, atom_name_list)

# posit_pix_peaks is correccted posit_pix by peak_finders

np.save(base_dir + first_posit_pix_file_name + "_" + sample, posit_pix_peaks[0][0])


## Parameter setting for unit cell optimization

In [ ]:
# each element is for each atom in the keywords of Atom_positions_dic.

pad_size_list = [20, 20, 20]                               
amplitude = [5.0, 5.0, 5.0]       # Initial guess of Gaussian amplitude 
sig_x = [5.0, 5.0, 5.0]           # Initial guess of Gaussian width in x direction in pixel
sig_y = [5.0, 5.0, 5.0]           # Initial guess of Gaussian width in y direction in pixel 
theta = [0.0, 0.0, 0.0]           # Initial guess of roataion angle or Gaussian function in deg


tfv_dposit = tf.Variable(tf.zeros((2, atom_num_in_unit_cell), dtype = tf.float32))              
# modulation of atomic positions in the unit cell to be optimized  

tfv_unit_cell_params = tf.Variable(tf.stack([amplitude, sig_x, sig_y, theta], axis = 0))        
# Gaussian parameters of the unit cell to be optimized

loss_array = [] 

pad_size_list_file_name = "pad_size_list"
np.save(base_dir + pad_size_list_file_name + "_" + sample, pad_size_list)


## Unit cell optimization

In [ ]:
atom_positions_in_unit_cell_file_name = "atom_positions_in_unit_cell"       # Optimized atomic positions in unit cell to save (.npy)
unit_cell_params_file_name = "unit_cell_params"                             # Optimized unit cell Gaussian paramters to save (.npy)

n_iter = 300                 # number of interation of gradient decent

lr_pos = 1e-2                # learning rate of the positional modulation update (tfv.deposit)
lr_param = 1e-2              # learning rate of the Gaussian parameters update (tfv_unit_cell_params)

print_num = 10               # loss will be printed out each print_num'th iteration

Fixed_params = [[1, 1], [2, 2]]     # [params_index, atom_index]

Max_position_modul = 0.5     # Maximum amount of positional modulations in pixel

#------------------------------------------------------------------------------------------------------

tfv_dposit.assign(tf.clip_by_value(tfv_dposit, -Max_position_modul, Max_position_modul))

pos_optimizer = tf.optimizers.Adam(learning_rate=lr_pos) 
param_optimizer = tf.optimizers.Adam(learning_rate=lr_param)
variables = [tfv_dposit, tfv_unit_cell_params]

for i in range(n_iter):

    with tf.GradientTape() as tape:

        simulated_image = gf.Gaussian_unitcell(posit_pix_peaks, tfv_dposit, tfv_unit_cell_params, 
                                               atom_num_list, pad_size_list, unit_cell_image.shape)

        loss = lf.compute_loss_tf(tf_unit_cell_image, simulated_image)
        loss_array.append(loss)
  
        grads = tape.gradient(loss, variables) 

        if len(Fixed_params) != 0:
            grads[1] = tf.tensor_scatter_nd_update(grads[1], Fixed_params, 
                                                   tf.zeros((len(Fixed_params),)))
            
        pos_optimizer.apply_gradients([(grads[0], tfv_dposit)])
        param_optimizer.apply_gradients([(grads[1], tfv_unit_cell_params)])

        np.save(base_dir + atom_positions_in_unit_cell_file_name + "_" + sample, np.array(tfv_dposit))
        np.save(base_dir + unit_cell_params_file_name + "_" + sample, np.array(tfv_unit_cell_params))  

        if i % print_num == 0:
            
            print(f'{i}, current_loss: {loss}')
     
plt.plot(np.arange(len(loss_array)), loss_array)



## Unit cell optimization result

In [ ]:
atom_positions_in_unit_cell_file_name = "atom_positions_in_unit_cell"       # Optimized atomic poistions in unit cell to load (.npy)
unit_cell_params_file_name = "unit_cell_params"                             # Optimized unit cell Gaussian paramters to load (.npy)

unit_cell_information_file_name = "unit_cell_information"              # Unit cell information to save (.txt)

marker_size = 50

(tfv_dposit, tfv_unit_cell_params, atom_resolved_positions, atom_resolved_params, 
 unit_cell_simul_image, Unit_cell_markers, Unit_cell_comparison) = \
tools.unit_cell_optimization_result(posit_pix, atom_num_list, pad_size_list, unit_cell_image, unit_cell_image.shape, 
                                    atom_positions_in_unit_cell_file_name, unit_cell_params_file_name, sample, 
                                    atom_name_list, unit_cell_information_file_name, marker_size,
                                    base_dir)

Unit_cell_markers.savefig(base_dir + f"Unit_cell_markers_{sample}.png", dpi = 300, bbox_inches = 'tight')
Unit_cell_comparison.savefig(base_dir + f"Unit_cell_comparison_{sample}.png", dpi = 300, bbox_inches = 'tight')

np.savetxt(base_dir + f"Unit_cell_ndarray_{sample}.txt", unit_cell_image)
np.savetxt(base_dir + f"Simulated_unit_cell_ndarray_{sample}.txt", unit_cell_simul_image)

for atom in range(len(atom_num_list)):

    np.savetxt(base_dir + f"{atom_name_list[atom]}_x_unit_cell_{sample}.txt", atom_resolved_positions[atom][0, :])
    np.savetxt(base_dir + f"{atom_name_list[atom]}_y_unit_cell_{sample}.txt", atom_resolved_positions[atom][1, :])
    np.savetxt(base_dir + f"{atom_name_list[atom]}_A_unit_cell_{sample}.txt", atom_resolved_params[atom][0, :])
    np.savetxt(base_dir + f"{atom_name_list[atom]}_sigx_unit_cell_{sample}.txt", atom_resolved_params[atom][1, :])
    np.savetxt(base_dir + f"{atom_name_list[atom]}_sigy_unit_cell_{sample}.txt", atom_resolved_params[atom][2, :])
    np.savetxt(base_dir + f"{atom_name_list[atom]}_theta_unit_cell_{sample}.txt", atom_resolved_params[atom][3, :])

uc_result_dic = {}

for atom in range(len(atom_num_list)):

    uc_result_dic[atom_name_list[atom]+"_x"] = atom_resolved_positions[atom][0, :]
    uc_result_dic[atom_name_list[atom]+"_y"] = atom_resolved_positions[atom][1, :]
    uc_result_dic[atom_name_list[atom]+"_A"] = atom_resolved_params[atom][0, :]
    uc_result_dic[atom_name_list[atom]+"_sigx"] = atom_resolved_params[atom][1, :]
    uc_result_dic[atom_name_list[atom]+"_singy"] = atom_resolved_params[atom][2, :]
    uc_result_dic[atom_name_list[atom]+"_theta"] = atom_resolved_params[atom][3, :]

## Position generator

In [ ]:
# Run this code to see that the positions are well generated.

# positions is an array with the shape of (2, n), where 2 is for x and y cordinates and n is the number of total atoms analyzed.
# atom_resolved_positions is a list whose elements are the positions of each atom type.

positions_file_name = "positions"

positions, params = tools.positions_params_gen(lattices, posit_pix_peaks, tfv_dposit, 
                                               tfv_unit_cell_params, lattice_num, atom_num_list)
atom_resolved_positions = tools.unpack_atom_type(positions,  lattice_num,  atom_num_list)
# list of positions, where each list elements the positions of each atom species

plt.imshow(im_analyzed, cmap = "gray")
plt.title("Positions look vaild?")

for atom in range(len(atom_num_list)):
    
    plt.scatter(atom_resolved_positions[atom][0,:], atom_resolved_positions[atom][1,:], s = 1)

np.save(base_dir + positions_file_name + "_" + sample, positions) 

simulated_image = gf.Gaussian_np_fine(positions, np.zeros(positions.shape), 
                                        params, atom_num_list, 
                                        pad_size_list, im_shape)

print("loss from positions:", lf.compute_loss_np(im_analyzed, simulated_image))

## Lattice corrections

In [ ]:
anchor_atom = [0, 0]    # atom, indice of atom in posit_pix
anchor_atom_pad = 10

marker_size = 1

positions_c, lattices_c = tools.positions_correction(positions, posit_pix_peaks, 
                                               lattices, anchor_atom, anchor_atom_pad,
                                                     atom_num_list, lattice_num, 
                                               im_analyzed, marker_size, row, col, 
                                                    tfv_dposit)

atom_resolved_positions = tools.unpack_atom_type(positions_c, lattice_num, 
                                                 atom_num_list)

plt.figure()
plt.imshow(im_analyzed, cmap = "gray")
plt.title("Positions look vaild?")

for atom in range(len(atom_num_list)):
    
    plt.scatter(atom_resolved_positions[atom][0,:], atom_resolved_positions[atom][1,:], s = marker_size)

simulated_image = gf.Gaussian_np_fine(positions_c, np.zeros(positions.shape), 
                                        params, atom_num_list, 
                                        pad_size_list, im_shape)

print("loss from corrected positiions:", lf.compute_loss_np(im_analyzed, simulated_image))



## Position correction with peak finder

In [ ]:
positions_peak_file_name = "positions_peak"

peak_finder_pad_size_list = [5, 5, 5]
marker_size = 5


positions_peak = tools.positions_peak_finder(positions_c, 
                                                  atom_num_list, 
                                                  lattice_num, 
                                                         peak_finder_pad_size_list,
                                                        im_analyzed, atom_name_list)

atom_resolved_positions = tools.unpack_atom_type(positions_peak,  
                                                 lattice_num,  atom_num_list)

plt.figure()
plt.imshow(im_analyzed, cmap = "gray")
plt.title("Positions look vaild?")

for atom in range(len(atom_num_list)):
    
    plt.scatter(atom_resolved_positions[atom][0,:], atom_resolved_positions[atom][1,:], 
                s = marker_size)


tfv_dpositions = tf.Variable(tf.zeros((2, positions_peak.shape[1]), 
                                      dtype = tf.float32))
tfv_params = tf.Variable(tf.cast(params, dtype = tf.float32))

loss_array = []

np.save(base_dir + positions_peak_file_name + "_" + sample, positions_peak) 

simulated_image = gf.Gaussian_np_fine(positions_peak, np.zeros(positions.shape), 
                                        params, atom_num_list, 
                                        pad_size_list, im_shape)

print("\nloss from corrected positiions_peak:", lf.compute_loss_np(im_analyzed, simulated_image))

## Free atom optimization

In [ ]:
dpositions_file_name = "dpositions"         # Optimized positions
params_file_name = "params"                 # Optimized Gaussain parameters
loss_file_name = "loss"

n_iter = 500

lr_pos = 1e-3
lr_param = 1e-2

print_num = 10

Fixed_params = [[1, 1], [2, 2]]     

mask_params = tools.free_atom_mask(lattice_num, atom_num_list, Fixed_params)
# Gradient mask for fixed parameters

Max_position_modul = 0.5

#------------------------------------------------------------------------------------------------------

tfv_dpositions.assign(tf.clip_by_value(tfv_dpositions, -Max_position_modul, Max_position_modul))

im_shape = im_analyzed.shape

pos_optimizer = tf.optimizers.Adam(learning_rate=lr_pos) 
param_optimizer = tf.optimizers.Adam(learning_rate=lr_param)
variables = [tfv_dpositions, tfv_params]

for i in range(n_iter):

    with tf.GradientTape() as tape:

        simulated_image = gf.Gaussian_fine(positions_peak, tfv_dpositions, 
                                        tfv_params, atom_num_list, 
                                        pad_size_list, im_shape)

        loss = lf.compute_loss_tf(tf_im_analyzed, simulated_image)
        loss_array.append(loss)

        grads = tape.gradient(loss, variables) 

        if len(Fixed_params) != 0:

            grads[1] = tf.tensor_scatter_nd_update(grads[1], mask_params, 
                                                   tf.zeros([len(mask_params),]))
        
        pos_optimizer.apply_gradients([(grads[0], tfv_dpositions)])
        param_optimizer.apply_gradients([(grads[1], tfv_params)])
        
        np.save(base_dir + params_file_name + "_" + sample, np.array(tfv_params)) 
        np.save(base_dir + dpositions_file_name + "_" + sample, np.array(tfv_dpositions)) 

        if i % print_num == 0:
            
            print(f'{i}, current_loss: {loss}')
     
plt.plot(np.arange(len(loss_array)), loss_array)



## Free atom optimization results

In [ ]:
# Run this code to see the change in the lattice information and compare the TEM image and fitted image

# laod positions and params
dpositions_file_name = "dpositions"        # Optimized positions
params_file_name = "params"              # Optimized Gaussain parameters

marker_size = 5

positions_peak_file_name = "positions_peak"
lattice_num_file_name = "lattice_num"
atom_num_list_file_name = "atom_num_list"
pad_size_list_file_name = "pad_size_list"
im_analyzed_file_name = "im_analyzed"
atom_name_list_file_name = "atom_name_list"

np.save(base_dir + first_posit_pix_file_name + "_" + sample, posit_pix_peaks[0][0])


# positions and parameters are the arrays of optimized positions and Gaussian parameters with the shape of (2, n).
# tfv_params is the variable version of params.
# atom_resolved_positions is a list with the length of number of atom type, whose elements are the optmized positions of each atom type.

im_analyzed = np.load(base_dir + im_analyzed_file_name + "_" + sample + ".npy")
positions_peak = np.load(base_dir + positions_peak_file_name + "_" + sample + ".npy")
lattice_num = np.load(base_dir + lattice_num_file_name + "_" + sample + ".npy")
atom_num_list = np.load(base_dir + atom_num_list_file_name + "_" + sample + ".npy")
pad_size_list = np.load(base_dir + pad_size_list_file_name + "_" + sample + ".npy")
atom_name_list = np.load(base_dir + atom_name_list_file_name + "_" + sample + ".npy")

tf_im_analyzed = tf.cast(im_analyzed, dtype = tf.float32)

loss_array = []

tfv_params, tfv_dpositions, atom_resolved_positions, atom_resolved_params = \
tools.free_atom_opmization_results(base_dir, dpositions_file_name, params_file_name,
                                      positions_peak, lattice_num, atom_num_list, pad_size_list, 
                                      im_analyzed, marker_size, sample)



## Cutting boundary

In [ ]:
# Cutting boundaries. cut_col = 1 for cutting right and left most columns, and cut_row = 1 for cutting
# top and bottom rows.

cut_col = 1
cut_row = 0        

marker_size = 5

cut_atom_positions_file_name = "cut_atom_positions"
cut_atom_params_file_name = "cut_atom_params"

x_lattice_file_name = "x_lattice"
y_lattice_file_name = "y_lattice"
len_pix_file_name = "len_pix"

x_lattice = np.load(base_dir + x_lattice_file_name + "_" + sample + ".npy")
y_lattice = np.load(base_dir + y_lattice_file_name + "_" + sample + ".npy")
len_pix = np.load(base_dir + len_pix_file_name + "_" + sample + ".npy")

# Operation code -----------------------------------------------------------------------------------------------

# cut_atom_positions and cut_atom_params is the cut versions of positions and params.

cut_atom_positions, cut_atom_params = tools.boundary_cut(atom_resolved_positions, 
                                                         atom_resolved_params, 
                                                         row, col, cut_col = cut_col, 
                                                         cut_row = cut_row)

row_cut_start = int(y*cut_row/len_pix)
row_cut_end = im_analyzed.shape[0]-int(y*cut_row/len_pix)
col_cut_start = int(x*cut_col/len_pix)
col_cut_end = im_analyzed.shape[1]-int(x*cut_col/len_pix)

plt.imshow(im_analyzed[row_cut_start:row_cut_end, 
             col_cut_start : col_cut_end], cmap = "gray")
plt.title("TEM image with markers")

for atom in range(len(atom_num_list)):
    
    plt.scatter(cut_atom_positions[atom][0,:] - col_cut_start, 
                cut_atom_positions[atom][1,:]-row_cut_start, 
                s = marker_size, label = atom_name_list[atom])

plt.legend(loc = "lower left",
              bbox_to_anchor = (1.02, 0),
              frameon=False)
plt.savefig(base_dir + f"TEM image with markers_{sample}.png", 
            bbox_inches = "tight")

fig, ax = plt.subplots(1, 2, figsize = (10, 5))

ax[0].imshow(im_analyzed[row_cut_start:row_cut_end, 
             col_cut_start : col_cut_end], cmap = "gray")
ax[0].set_title("TEM image")
ax[1].imshow(gf.Gaussian_np_fine(positions_peak, tfv_dpositions, tfv_params, 
                                 atom_num_list, pad_size_list, im_analyzed.shape)[row_cut_start:row_cut_end, 
             col_cut_start : col_cut_end], cmap = "gray")
ax[1].set_title("Simulated TEM image")

cut_image = im_analyzed[row_cut_start:row_cut_end, col_cut_start : col_cut_end]
cut_simul = gf.Gaussian_np_fine(positions_peak, tfv_dpositions, tfv_params, 
                                atom_num_list, pad_size_list, im_analyzed.shape)[row_cut_start:row_cut_end, 
             col_cut_start : col_cut_end]

plt.savefig(base_dir + f"TEM image and fitted image_{sample}.png", bbox_inches = "tight")
np.savetxt(base_dir + f"TEM image for markers_{sample}.png", cut_image)


obj_positions = np.empty(len(cut_atom_positions), dtype=object)
for i, v in enumerate(cut_atom_positions):
    obj_positions[i] = np.array(v, copy=True)

obj_params = np.empty(len(cut_atom_params), dtype=object)
for i, v in enumerate(cut_atom_params):
    obj_params[i] = np.array(v, copy=True)

np.save(base_dir + cut_atom_positions_file_name + "_" + sample,
       obj_positions, allow_pickle=True)

np.save(base_dir + cut_atom_params_file_name + "_" + sample,
       obj_params, allow_pickle=True)

## Dictionary generation

In [ ]:
cut_atom_positions_file_name = "cut_atom_positions"
cut_atom_params_file_name = "cut_atom_params"
first_posit_pix_file_name = "first_posit_pix"
unitcell_file_name = "unitcell"
atom_num_list_file_name = "atom_num_list"
pad_size_list_file_name = "pad_size_list"
atom_name_list_file_name = "atom_name_list"

obj_positions = np.load(base_dir + cut_atom_positions_file_name +\
                        "_" + sample + ".npy", allow_pickle=True)
cut_atom_positions = obj_positions.tolist()  

obj_params = np.load(base_dir + cut_atom_params_file_name +\
                        "_" + sample + ".npy", allow_pickle=True)
cut_atom_params = obj_params.tolist()  

first_posit_pix = np.load(base_dir + first_posit_pix_file_name + "_" + sample + ".npy")
unit_cell_image = np.load(base_dir + unitcell_file_name + "_" + sample + ".npy")
atom_num_list = np.load(base_dir + atom_num_list_file_name + "_" + sample + ".npy")
pad_size_list = np.load(base_dir + pad_size_list_file_name + "_" + sample + ".npy")
atom_name_list = np.load(base_dir + atom_name_list_file_name + "_" + sample + ".npy")

dictionary = {}

dictionary["posit_pix_optimized"] = []
dictionary["unit_cell_params_optimized"] = []

for atom in range(len(atom_num_list)):

    dictionary[atom_name_list[atom] + "_pos"] = cut_atom_positions[atom]
    dictionary[atom_name_list[atom] + "_par"] = cut_atom_params[atom]

    dictionary[atom_name_list[atom] + "_pos_avg"] = np.mean(dictionary[atom_name_list[atom] + "_pos"], axis = 1)
    dictionary[atom_name_list[atom] + "_par_avg"] = np.mean(dictionary[atom_name_list[atom] + "_par"], axis = 1)

    dictionary[atom_name_list[atom] + "_pos_std"] = np.std(dictionary[atom_name_list[atom] + "_pos"], axis = 1)
    dictionary[atom_name_list[atom] + "_par_std"] = np.std(dictionary[atom_name_list[atom] + "_par"], axis = 1)

    positions_aligned = \
    cut_atom_positions[atom].reshape(2, -1, atom_num_list[atom])

    params_aligned = \
    cut_atom_params[atom].reshape(4, -1, atom_num_list[atom])

    dictionary[atom_name_list[atom] + "_uc_pos"] =\
    np.mean(positions_aligned,
           axis = 2)

    dictionary[atom_name_list[atom] + "_uc_par"] =\
    np.mean(params_aligned,
           axis = 2)

    for order in range(atom_num_list[atom]):

        dictionary[atom_name_list[atom]+ f"_{order}" + "_pos"] = positions_aligned[:, :, order]
        dictionary[atom_name_list[atom]+ f"_{order}" + "_par"] = params_aligned[:, :, order]

        dictionary[atom_name_list[atom]+ f"_{order}" + "_pos_avg"] = np.mean(dictionary[atom_name_list[atom]+ f"_{order}" + "_pos"], axis = 1)
        dictionary[atom_name_list[atom]+ f"_{order}" + "_par_avg"] = np.mean(dictionary[atom_name_list[atom]+ f"_{order}" + "_par"], axis = 1)

        dictionary[atom_name_list[atom]+ f"_{order}" + "_pos_std"] = np.std(dictionary[atom_name_list[atom]+ f"_{order}" + "_pos"], axis = 1)
        dictionary[atom_name_list[atom]+ f"_{order}" + "_par_std"] = np.std(dictionary[atom_name_list[atom]+ f"_{order}" + "_par"], axis = 1)

for atom in range(len(atom_num_list)):

    np.savetxt(base_dir + atom_name_list[atom] + "_pos" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_pos"])

    np.savetxt(base_dir + atom_name_list[atom] + "_par" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_par"])

    np.savetxt(base_dir + atom_name_list[atom] + "_pos_avg" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_pos_avg"])

    np.savetxt(base_dir + atom_name_list[atom] + "_par_avg" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_par_avg"])

    np.savetxt(base_dir + atom_name_list[atom] + "_pos_std" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_pos_std"])

    np.savetxt(base_dir + atom_name_list[atom] + "_par_std" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_par_std"])# atom_par_avg(std) : Gaussian paramters of the atom

    np.savetxt(base_dir + atom_name_list[atom] + "_uc_pos" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_uc_pos"])

    np.savetxt(base_dir + atom_name_list[atom] + "_uc_par" + "_" + sample, 
               dictionary[atom_name_list[atom] + "_uc_par"])

    for order in range(atom_num_list[atom]):

        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_pos" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_pos"])
        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_par" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_par"])
        
        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_pos_avg" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_pos_avg"])
        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_par_avg" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_par_avg"])
        
        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_pos_std" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_pos_std"])
        np.savetxt(base_dir + atom_name_list[atom]+ f"_{order}" + "_par_std" + "_" + sample, 
           dictionary[atom_name_list[atom]+ f"_{order}" + "_par_std"])


# atom_pos(par) : positions (Gaussian parameters) of the atom

# atom_pos(par)_avg(std) : average(standard deviation) of atom_pos(par)

# atom_uc_pos(par) : unit cell average positions(Gaussian parameters) of the atom

# atom_n_pos(par) : positions(Gaussain parameters) of the atom at n'th position in the unit cell

# atom_n_pos(par)_avg(std) : average(standard deviation) of atom_pos(par) of atom_n_pos(par)

